In [1]:
import os
import langchain
import langchain_openai
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAI
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain 
from langchain.memory import ConversationBufferMemory
from langchain.utilities import WikipediaAPIWrapper 

In [2]:
#RAG related imports
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings


In [3]:
from langchain.chains.question_answering import load_qa_chain
from langchain.callbacks import get_openai_callback


In [4]:
from langchain_community.chat_models import ChatOpenAI
from langchain_community.embeddings import CohereEmbeddings
from langchain_community.vectorstores import Pinecone
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from pinecone import Pinecone as PineconeClient
import requests

In [36]:
from langchain.retrievers.multi_query import MultiQueryRetriever


In [5]:
#Your Pinecone API key.
os.environ['PINECONE_API_KEY']= "pinecone api key"   
#The name of the index you want to use.
os.environ['PINECONE_INDEX_NAME'] ="https://ds-index-dqr5yj5.svc.gcp-starter.pinecone.io"
os.environ["PINECONE_ENVIRONMENT"]="gcp-starter"

In [6]:
PINECONE_API_KEY = os.environ["PINECONE_API_KEY"]
PINECONE_ENVIRONMENT = os.environ["PINECONE_ENVIRONMENT"]
PINECONE_INDEX_NAME = os.environ["PINECONE_INDEX_NAME"]


In [7]:
 os.environ["PINECONE_INDEX_NAME"]

'https://ds-index-dqr5yj5.svc.gcp-starter.pinecone.io'

In [8]:
from langchain.prompts import ChatPromptTemplate

In [9]:
from langchain_community.embeddings import HuggingFaceInstructEmbeddings

In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.chains import RetrievalQA

from langchain.document_loaders import DirectoryLoader

In [11]:
#import open AI api key.
os.environ['OPENAI_API_KEY']="open api key"

In [12]:
#Used to load pdfs
from langchain_community.document_loaders import PyPDFLoader


In [13]:
from platform import python_version

print(python_version())

3.11.5


Loading and splitting the Documents into chunks

In [14]:

loader = PyPDFLoader("Dsa.pdf")
pages = loader.load_and_split()

In [15]:
print(pages[1])

page_content='Data Structures and Algorithms:\nAnnotated Reference with Examples\nFirst Edition\nCopyright c⃝Granville Barnett, and Luca Del Tongo 2008.' metadata={'source': 'Dsa.pdf', 'page': 1}


In [16]:
pages[0]

Document(page_content='DSADat a St ruc tur es and Alg orith ms\nAnn otated Reference with Examp les\nGranville Bar ne/g425 Luca  Del Tongo', metadata={'source': 'Dsa.pdf', 'page': 0})

In [28]:
from langchain_community.vectorstores import FAISS

faiss_index0 = FAISS.from_documents(pages, model_norm)


In [33]:
docs0 = faiss_index0.similarity_search("what are advantages of DSA?", k=2)


In [34]:
print(docs0)

[Document(page_content='Chapter 2\nLinked Lists\nLinked lists can be thought of from a high level perspective as being a series\nof nodes. Each node has at least a single pointer to the next node, and in the\nlast node’s case a null pointer representing that there are no more nodes in the\nlinked list.\nIn DSA our implementations of linked lists always maintain head and tail\npointers so that insertion at either the head or tail of the list is a constant\ntime operation. Random insertion is excluded from this and will be a linear\noperation. As such, linked lists in DSA have the following characteristics:\n1.Insertion is O(1)\n2.Deletion is O(n)\n3.Searching is O(n)\nOut of the three operations the one that stands out is that of insertion. In\nDSA we chose to always maintain pointers (or more aptly references) to the\nnode(s) at the head and tail of the linked list and so performing a traditional\ninsertion to either the front or back of the linked list is an O(1) operation. An\nexcept

In [44]:
model = ChatOpenAI(temperature=0)
retriever = MultiQueryRetriever.from_llm(retriever=faiss_index0.as_retriever(), llm=model)

In [45]:
query="what are advantages of DSA?"

In [53]:
docs = faiss_index0.similarity_search(query=query, k=3)
 
chain = load_qa_chain(llm=model, chain_type="stuff")
with get_openai_callback() as cb:
    response = chain.run(input_documents=docs, question=query)
print(cb)

Tokens Used: 1905
	Prompt Tokens: 1584
	Completion Tokens: 321
Successful Requests: 1
Total Cost (USD): $0.003018


In [54]:
print(response)

The advantages of DSA (Data Structures and Algorithms) include:

1. Efficiency: DSA allows for efficient storage and retrieval of data. By choosing appropriate data structures and implementing efficient algorithms, DSA can help optimize the performance of programs.

2. Flexibility: DSA provides a wide range of data structures and algorithms that can be used to solve a variety of problems. This flexibility allows programmers to choose the most suitable approach for their specific needs.

3. Reusability: DSA promotes code reusability. Once implemented, data structures and algorithms can be reused in different programs or projects, saving time and effort.

4. Scalability: DSA enables programs to handle large amounts of data effectively. By using efficient data structures and algorithms, DSA can handle increasing data sizes without significant performance degradation.

5. Problem-solving: DSA provides a systematic approach to problem-solving. By analyzing problems and selecting appropriate

In [60]:
# RAG prompt
template = """Answer the question based only on the following context:
{context}
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

# RAG
model = ChatOpenAI()
chain1 = ( RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
| prompt
| model
| StrOutputParser()
)

In [52]:
chain1.invoke(query)


AIMessage(content='Based on the given context, the advantages of DSA (Data Structures and Algorithms) are:\n\n1. Dynamic resizing: Linked lists, a type of data structure in DSA, can dynamically resize without incurring a copy penalty like an array or vector would eventually incur.\n\n2. Constant time insertion: Insertion at either the head or tail of a linked list in DSA is a constant time operation (O(1)) because the data structure maintains head and tail pointers.\n\n3. Fast look-up times: Sets implemented in DSA have fast look-up times. Unordered sets use a hash table for near constant run time complexity, while ordered sets use a binary search tree with logarithmic run time complexity.\n\n4. Efficient queue implementation: DSA provides an efficient way to implement a queue data structure using a singly linked list. Insertion and deletion in a queue implemented with a singly linked list have O(1) run time complexities.\n\n5. Priority queue support: DSA provides support for priority 

In [17]:
loader1 = PyPDFLoader("ctax.pdf")
pages1 = loader1.load_and_split()

In [18]:
pages1[0]

Document(page_content="Chapter 2  \nOverview of Australia's corporate tax system  \n2.1 This chapter provides an overview of Australia's corporate tax system and its \nimportance as a source of public revenue. In particular, this chapter:  \n• provides an introduction to Australia's corporate tax system;  \n• considers the international context; and  \n• explores the broad impacts associated with tax avoidance and aggressive \nminimisation.   \nIntroduction to Australia's corporate income tax system   \n2.2 Australia's taxation system is extremely complex and issues relating to  \ncorporate tax are no different. As such, the information presented in this section \nprovides a brief overview of how the corporate income tax system operates—noting \nthere are many highly technical  rules and interpretations that affect businesses  and \ninvestment decisions, and the amount of corporate income tax paid.   \n2.3 While corporations pay a variety of taxes and levies (including payroll tax, \nG

In [19]:
print(len(pages))
len(pages1)

112


10

In [20]:
documents=loader.load()

In [21]:
print(documents[1])

page_content='Data Structures and Algorithms:\nAnnotated Reference with Examples\nFirst Edition\nCopyright c⃝Granville Barnett, and Luca Del Tongo 2008.' metadata={'source': 'Dsa.pdf', 'page': 1}


In [22]:
documents1=loader1.load()

In [23]:
print(documents1[1])

page_content="8  \n 2.7 Where assessable income is derived from activities outside Australia, that \nincome is generally exempt from corporate income tax provided that the income was \n'actively' earned.2 In cer tain circumstances, however, corporations may be required to \npay 'top- up' tax to Australia on repatriated earnings as required by Controlled Foreign \nCompany rules. For example, BHP Billiton has paid 'top-up' tax to Australia on profits \nrepatriated from its Singapore marketing hub.3 \nCertain types of corporations are exempt from paying income tax  \n2.8 While most corporations undertaking business activities are required to pay \ncorporate income tax, certain types of  corporate entities are not liable.  \n2.9 Partnerships and trusts are not required to pay corporate tax provided their \nassessable income is distributed to unit holders. Unit holders are then required to pay \neither corporate tax (in the case of a company) or personal income tax (in the case of \nindivid

NameError: name 'documents_dir' is not defined

In [25]:
#pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [27]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-base-en"
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity

model_norm = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    encode_kwargs=encode_kwargs
)

#I used BHE lite model, that has 768 dimensions as embedding model, faiss index ad index storage and vectordb

In [29]:
from langchain_community.vectorstores import FAISS

faiss_index = FAISS.from_documents(pages1, model_norm)


In [30]:
docs12 = faiss_index.similarity_search("what are different Australia corporate tax types?", k=2)


In [31]:
for doc in docs12:
    print(str(doc.metadata["page"]) + ":", doc.page_content[:300])

0: Chapter 2  
Overview of Australia's corporate tax system  
2.1 This chapter provides an overview of Australia's corporate tax system and its 
importance as a source of public revenue. In particular, this chapter:  
• provides an introduction to Australia's corporate tax system;  
• considers the int
6: 13 
 A$945 million in 'top- up tax' to the ATO on the profits of BHP Billiton Marketi ng 
(Singapore Branch) for the period 2006 to 2014.28  
Treatment of income sourced in Australia by foreign-based organisations  
2.39 Where tax treaties exist between jurisdictions, companies can effectively 
choo


#Used OpenAi model as LLm.

In [37]:
model = ChatOpenAI(temperature=0)
retriever = MultiQueryRetriever.from_llm(retriever=faiss_index.as_retriever(), llm=model)

In [ ]:
# RAG prompt
template = """Answer the question based only on the following context:
{context}
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

# RAG
model = ChatOpenAI()
chain1 = (
RunnableParallel({"context": retriever0, "question": RunnablePassthrough()})
| prompt
| model
| StrOutputParser()
)

In [ ]:
chain1.invoke("what are different corporate taxes in Australia?")


In [ ]:
chain1.invoke("what is difference between income tax and payroll tax in Australia?")


In [ ]:
chain123.invoke("what are different DSA advantages?")


In [ ]:
from langchain_community.vectorstores import FAISS

faiss_index1 = FAISS.from_documents(pages1, OpenAIEmbeddings())
docs123 = faiss_index1.similarity_search("what are different Australia corporate tax types?", k=2)
for doc in docs123:
    print(str(doc.metadata["page"]) + ":", doc.page_content[:500])

openai embeddings with open ai llm

In [ ]:
model1 = ChatOpenAI(temperature=0)
retriever1 = MultiQueryRetriever.from_llm(retriever=faiss_index1.as_retriever(), llm=model1)

In [ ]:
# RAG prompt
template = """Answer the question based only on the following context:
{context}
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

# RAG
chain12 = (
RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
| prompt
| model1
| StrOutputParser()
)

In [ ]:
chain12.invoke("what are different corporate taxes in Australia?")


In [ ]:
chain12.invoke("what is difference between income tax and payroll tax in Australia?")


In [ ]:
from langchain_community.vectorstores import FAISS

faiss_index1 = FAISS.from_documents(pages, OpenAIEmbeddings())
docs1 = faiss_index1.similarity_search("what are DSA advantages?", k=2)
for doc in docs1:
    print(str(doc.metadata["page"]) + ":", doc.page_content[:500])

In [ ]:
model1 = ChatOpenAI(temperature=0)
retriever = MultiQueryRetriever.from_llm(retriever=faiss_index1.as_retriever(), llm=model1)

In [ ]:
# RAG prompt
template = """Answer the question based only on the following context:
{context}
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

# RAG
chain_1 = (
RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
| prompt
| model1
| StrOutputParser()
)

In [ ]:
chain_1.invoke("what are different DSA advantages?")


#pine cone and cohere API values
#Pinecone is used to store the vector indices and Openai got multiple embedding models, in here i chose 1536 emmbedding model, (Can experiement with 1024,768 embedding model)

In [ ]:
#using openai embedding and storing in pinecone vector store

In [ ]:
from langchain.vectorstores import Pinecone as PineconeStore

In [ ]:

index = pinecone.Index("ds-index")

index.describe_index_stats()


In [ ]:

loader33 = PyPDFLoader("ctax.pdf")
data33 = loader33.load()
# Split your data up into smaller documents with Chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(data33)
# Convert Document objects into strings
pages33 = [str(doc) for doc in documents]

In [ ]:

loader44 = PyPDFLoader("Dsa.pdf")
data44 = loader44.load()
# Split your data up into smaller documents with Chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(data44)
# Convert Document objects into strings
pages44 = [str(doc) for doc in documents]

In [ ]:
index_name="ds-index"

In [ ]:
# Install the latest version
#!pip install pinecone-client



In [ ]:
# Install the latest version, with extra grpc dependencies
#!pip install "pinecone-client[grpc]"


In [ ]:

# Install a specific version
#!pip install pinecone-client==3.0.0

# Install a specific version, with grpc extras
#!pip install "pinecone-client[grpc]"==3.0.0

In [ ]:
import os
from pinecone.grpc import PineconeGRPC

pc = PineconeGRPC(api_key=os.environ.get('PINECONE_API_KEY'))

# From here on, everything is identical to the REST-based client.
index = pc.Index(host='https://ds-index-dqr5yj5.svc.gcp-starter.pinecone.io')

In [ ]:
from pinecone import Pinecone
pc = Pinecone(api_key=PINECONE_API_KEY)

In [ ]:
from langchain.vectorstores import Pinecone
from langchain.embeddings import OpenAIEmbeddings
import pinecone

embeddings = OpenAIEmbeddings()
my_pinecone = Pinecone.from_texts(
    pages33,  # an iterable of strings you want to build your Pinecone index with
    embeddings,
    index_name="ds-index")  # provide your own unique index_name

In [ ]:
from pinecone import Pinecone

index_description = pc.describe_index("ds-index")

In [ ]:
print(index_description)

In [ ]:
vectorstore = my_pinecone 
model = ChatOpenAI(temperature=0)
retriever234 = MultiQueryRetriever.from_llm(retriever=vectorstore.as_retriever(), llm=model)

In [ ]:
retriever234

In [ ]:
# RAG prompt
template = """Answer the question based only on the following context:
{context}
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

# RAG
model = ChatOpenAI()

chain123 = (
    RunnableParallel({"context": retriever234, "question": RunnablePassthrough()})
    | prompt
    | model
    | StrOutputParser()
)

In [ ]:
chain123.invoke("what are different Australia corporate tax types?")

In [ ]:
chain123.invoke("what are different Australia payroll tax types?")

In [ ]:
doc4 = my_pinecone.similarity_search("what are different Australia corporate tax types?", k=2)


In [ ]:
print(doc4)

In [ ]:
chain123.invoke("what is DSA difficuluties?")


In [ ]:
from langchain.vectorstores import Pinecone
from langchain.embeddings import OpenAIEmbeddings
import pinecone

embeddings = OpenAIEmbeddings()
my_pinecone = Pinecone.from_texts(
    pages44,  # an iterable of strings you want to build your Pinecone index with
    embeddings,
    index_name="ds-index")  # provide your own unique index_name

In [ ]:
from pinecone import Pinecone

index_description = pc.describe_index("ds-index")
print(index_description)

In [ ]:
vectorstore = my_pinecone 
model = ChatOpenAI(temperature=0)
retriever2345 = MultiQueryRetriever.from_llm(retriever=vectorstore.as_retriever(), llm=model)

In [ ]:
# RAG prompt
template = """Answer the question based only on the following context:
{context}
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

# RAG
model = ChatOpenAI()

chain1234 = (
    RunnableParallel({"context": retriever2345, "question": RunnablePassthrough()})
    | prompt
    | model
    | StrOutputParser()
)

In [ ]:
chain123.invoke("what is DSA difficuluties?")


In [ ]:
chain123.invoke("what are different DSA advantages?")


In [ ]:
chain123.invoke("tell me about DSA ?")


In [ ]:
doc5 = my_pinecone.similarity_search("what are DSA advantages?", k=2)
print(doc5)

In [ ]:
chain1234.invoke("tell me about DSA ?")


In [ ]:
chain1234.invoke("what is DSA difficuluties?")


In [ ]:
chain1234.invoke("what are  DSA advantages?")
